In [108]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats

from sklearn import preprocessing

In [109]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [110]:
def get_x_y(df):
  if 'class' in df.columns:
    df_x = df.drop(columns=['id','class'])
    df_y = df['class']
    return df_x, df_y
  else :
    df_x = df.drop(columns=['id'])
    return df_x

In [111]:
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [112]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

In [113]:
train_x1 = pd.get_dummies(train_x)

In [114]:
test_x1 = pd.get_dummies(test_x)

In [115]:
sampler = SMOTE()
sampler.fit(train_x1, train_y)

SMOTE()

In [116]:
X_over, Y_over = sampler.fit_resample(train_x1, train_y)

In [117]:
pipe_list = [('scaler',MinMaxScaler()),
             ('model',SVC())]
pip_model = Pipeline(pipe_list)

In [118]:
hyper_parameter = {'model__C':[100],
                  'model__kernel':['linear'],
                  'model__class_weight':[None, 'balanced']}

grid_model = GridSearchCV(pip_model, 
                         param_grid=hyper_parameter, 
                         cv=5, n_jobs = -1, scoring='f1')

grid_model.fit(train_x1, train_y)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('model', SVC())]),
             n_jobs=-1,
             param_grid={'model__C': [100],
                         'model__class_weight': [None, 'balanced'],
                         'model__kernel': ['linear']},
             scoring='f1')

In [119]:
best_model = grid_model.best_estimator_
train_y_pred = best_model.predict(train_x1)
test_y_pred  = best_model.predict(test_x1)

In [120]:
print(classification_report(train_y,train_y_pred))

              precision    recall  f1-score   support

           A       1.00      1.00      1.00        69
           B       1.00      1.00      1.00       114
           C       1.00      1.00      1.00        79

    accuracy                           1.00       262
   macro avg       1.00      1.00      1.00       262
weighted avg       1.00      1.00      1.00       262



In [121]:
test_y_pred = pd.DataFrame(test_y_pred)
test_y_pred= test_y_pred.replace({0:'A',1:'B',2:'C'})
test_y_pred.value_counts()

B    86
A    51
C    38
dtype: int64

In [122]:
test_y_pred['id'] = test['id']
test_y_pred['class'] = test_y_pred[0]

In [123]:
test_y_pred = test_y_pred.drop(columns=[0])
test_y_pred = test_y_pred.set_index('id')

In [124]:
test_y_pred.to_csv('submitNotsamleSVM.csv',mode='w')